## Libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
import geopandas as gdp
import shapely
import json
from bson.json_util import dumps

## Converting  Pandas Dataframe to a MongoDB Geopoint Object

In [2]:
def create_geoloc(path_csv):
    '''
    lo que hace
    '''
    
    #recibe el dataframe
    df =pd.read_csv(path_csv)
    
    #Crea el loc con los geopuntos.
    gdf = gdp.GeoDataFrame(df, geometry= gdp.points_from_xy(df.longitud, df.latitud ))
    gdf.columns=['name', 'lat','long', 'category','loc']
    
    #Lo aplicamos a toda la columna
    gdf['loc']= gdf['loc'].apply(lambda x:shapely.geometry.mapping(x))
   
    return gdf
    

### Bilbao collection

In [16]:
gdf_bilbao = create_geoloc('bilbao.csv')

In [17]:
gdf_bilbao.head()

,name,lat,long,category,loc
0,La Camelia Vegan-Bar,43.261577,-2.925420,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-2.92542, 43..."
1,Bohemian Lane,43.256430,-2.924784,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-2.924784121..."
2,Garibolo,43.258353,-2.933280,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-2.933279752..."
3,Restaurante Luum,43.264866,-2.933099,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-2.933099, 4..."
4,Areatza - El Arenal,43.260141,-2.923918,'Park',"{'type': 'Point', 'coordinates': (-2.923918305..."


In [18]:
client = MongoClient()
db = client.geo
collection_bilbao = db.bilbao
collection_bilbao.create_index([("loc", "2dsphere")])

data_bilbao = gdf_bilbao.to_dict(orient='records')
collection_bilbao.insert_many(data_bilbao)

### Gijon collection

In [22]:
gdf_gijon = create_geoloc("gijon.csv")

In [23]:
gdf_gijon.head()

,name,lat,long,category,loc
0,Escuela Infantil Municipal “Ana María Matute”,39.861411,-3.947029,'Preschool',"{'type': 'Point', 'coordinates': (-3.947028815..."
1,Madre Tierra,39.856617,-4.022343,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-4.022342701..."
2,Fruteria Santos,39.940697,-3.991567,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-3.991566896..."
3,Frutas Hermanos Farelo,39.962243,-4.194303,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-4.194303454..."
4,Parque De La Vega,39.863761,-4.024856,'Park',"{'type': 'Point', 'coordinates': (-4.024855771..."


In [24]:
client = MongoClient()
db = client.geo
collection_gijon = db.gijon
collection_gijon.create_index([("loc", "2dsphere")])

data_gijon = gdf_gijon.to_dict(orient='records')
collection_gijon.insert_many(data_gijon)

###  Madrid collection

In [4]:
gdf_madrid = create_geoloc('madrid.csv')

In [5]:
gdf_madrid.head()

,name,lat,long,category,loc
0,ABC Centre d'Estudis,40.402726,-3.674894,'Preschool',"{'type': 'Point', 'coordinates': (-3.6748938, ..."
1,Guardería Chiqui Tin Moratalaz,40.409716,-3.653979,'Preschool',"{'type': 'Point', 'coordinates': (-3.653978810..."
2,Foodies in the City,40.421212,-3.717058,'Preschool',"{'type': 'Point', 'coordinates': (-3.717057899..."
3,Planet Organic Food&Life,40.428808,-3.681333,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-3.681333, 4..."
4,Swagat Indian Restaurant,40.424513,-3.683114,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-3.683114294..."


In [6]:
client = MongoClient()
db = client.geo
collection_madrid = db.madrid
collection_madrid.create_index([("loc", "2dsphere")])

data_madrid = gdf_madrid.to_dict(orient='records')
collection_madrid.insert_many(data_madrid)

###  Toledo collection

In [28]:
gdf_toledo = create_geoloc('toledo.csv')

In [29]:
gdf_toledo.head()

,name,lat,long,category,loc
0,Escuela Infantil Municipal “Ana María Matute”,39.861411,-3.947029,'Preschool',"{'type': 'Point', 'coordinates': (-3.947028815..."
1,Madre Tierra,39.856617,-4.022343,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-4.022342701..."
2,Fruteria Santos,39.940697,-3.991567,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-3.991566896..."
3,Frutas Hermanos Farelo,39.962243,-4.194303,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': (-4.194303454..."
4,Parque De La Vega,39.863761,-4.024856,'Park',"{'type': 'Point', 'coordinates': (-4.024855771..."


In [30]:
client = MongoClient()
db = client.geo
collection_toledo = db.toledo
collection_toledo.create_index([("loc", "2dsphere")])

data_toledo = gdf_toledo.to_dict(orient='records')
collection_toledo.insert_many(data_toledo)

## GeoQueries

### Creating the queries

In [10]:
def queries_close (collection, lon, lat):
    query = [{
    "$geoNear": {'near': [lon, lat],
                 'distanceField': 'distance',
                 'maxDistance': 7000,
                 'distanceMultiplier': 6371, 
                 'spherical'  : True}}]
    geoloc = collection.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    
    return response_json
    

In [11]:
def check_results(json):
    results_list = list(json)
    
    if len(results_list) > 0:
        response = {"number of registers" : len(results_list),
                    "datos" : json}
        return (f"Nice! we have found  {len(results_list)}  results")
    else:
        return "Sorry, wa can´t find your petition"
        
        

#### Bilbao

In [19]:
bilbao = queries_close(collection_bilbao, -2.9253, 43.2627)
bilbao_check= check_results(bilbao)

In [20]:
bilbao_check

'Nice! we have found  78  results'

In [21]:
df_bilbao = pd.DataFrame(bilbao)
df_bilbao.head()

,_id,name,lat,long,category,loc,distance
0,{'$oid': '6020223c70aebdade83ada62'},La Camelia Vegan-Bar,43.261577,-2.925420,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-2.92542, 43...",0.125249
1,{'$oid': '60202af8b1c4f08d4af2c8a2'},La Camelia Vegan-Bar,43.261577,-2.925420,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-2.92542, 43...",0.125249
2,{'$oid': '6020223c70aebdade83ada77'},NBA Fan Zone,43.261363,-2.923943,'Basketball',"{'type': 'Point', 'coordinates': [-2.923942690...",0.184876
3,{'$oid': '60202af8b1c4f08d4af2c8b7'},NBA Fan Zone,43.261363,-2.923943,'Basketball',"{'type': 'Point', 'coordinates': [-2.923942690...",0.184876
4,{'$oid': '6020223c70aebdade83ada78'},Mundobasket Fan Zone,43.260412,-2.924411,'Basketball,"{'type': 'Point', 'coordinates': [-2.924410901...",0.264374


#### Gijon

In [25]:
gijon = queries_close(collection_gijon, -5.7258, 43.5317)
gijon_check= check_results(gijon)

In [26]:
gijon_check

'Nice! we have found  48  results'

In [27]:
df_gijon = pd.DataFrame(gijon)
df_gijon.head()

,_id,name,lat,long,category,loc,distance
0,{'$oid': '6020262970aebdade83ada9a'},Polideportivo El nido,40.240575,-3.777031,'Basketball Court',"{'type': 'Point', 'coordinates': [-3.777031271...",399.904299
1,{'$oid': '60202b59b1c4f08d4af2c8da'},Polideportivo El nido,40.240575,-3.777031,'Basketball Court',"{'type': 'Point', 'coordinates': [-3.777031271...",399.904299
2,{'$oid': '6020262970aebdade83ada95'},Parque Del Pilar,40.054756,-3.992189,'Park',"{'type': 'Point', 'coordinates': [-3.992189, 4...",412.440290
3,{'$oid': '60202b59b1c4f08d4af2c8d5'},Parque Del Pilar,40.054756,-3.992189,'Park',"{'type': 'Point', 'coordinates': [-3.992189, 4...",412.440290
4,{'$oid': '6020262970aebdade83ada96'},Parque De Yuncos,40.087130,-3.875277,'Park',"{'type': 'Point', 'coordinates': [-3.875276818...",412.555056


####  Madrid

In [12]:
madrid = queries_close(collection_madrid, -3.6793, 40.42955)
madrid_check= check_results(madrid)

In [13]:
madrid_check

'Nice! we have found  230  results'

In [14]:
df_madrid = pd.DataFrame(madrid)
df_madrid.head()

,_id,name,lat,long,category,loc,distance
0,{'$oid': '602029c970aebdade83adaa6'},Planet Organic Food&Life,40.428808,-3.681333,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-3.681333, 4...",0.190836
1,{'$oid': '60202a6fb1c4f08d4af2c831'},Planet Organic Food&Life,40.428808,-3.681333,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-3.681333, 4...",0.190836
2,{'$oid': '602029c970aebdade83adac9'},Pasillo de Diego de León,40.432397,-3.674888,'Platform',"{'type': 'Point', 'coordinates': [-3.674888329...",0.489569
3,{'$oid': '60202a6fb1c4f08d4af2c854'},Pasillo de Diego de León,40.432397,-3.674888,'Platform',"{'type': 'Point', 'coordinates': [-3.674888329...",0.489569
4,{'$oid': '602029c970aebdade83adaab'},Fit Food Velázquez 39,40.426034,-3.684132,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-3.684132099...",0.565812


####  Toledo

In [31]:
toledo = queries_close(collection_toledo, -3.6793, 40.42955)
toledo_check= check_results(toledo)

In [32]:
toledo_check

'Nice! we have found  24  results'

In [33]:
df_toledo = pd.DataFrame(toledo)
df_toledo.head()

,_id,name,lat,long,category,loc,distance
0,{'$oid': '60202d40b1c4f08d4af2c8f3'},Polideportivo El nido,40.240575,-3.777031,'Basketball Court',"{'type': 'Point', 'coordinates': [-3.777031271...",22.586977
1,{'$oid': '60202d40b1c4f08d4af2c8ef'},Parque De Yuncos,40.087130,-3.875277,'Park',"{'type': 'Point', 'coordinates': [-3.875276818...",41.548656
2,{'$oid': '60202d40b1c4f08d4af2c8f2'},Pabellón de deportes de Yuncos,40.081642,-3.873404,'Basketball Court',"{'type': 'Point', 'coordinates': [-3.873404, 4...",42.046315
3,{'$oid': '60202d40b1c4f08d4af2c8ee'},Parque Del Pilar,40.054756,-3.992189,'Park',"{'type': 'Point', 'coordinates': [-3.992189, 4...",49.417591
4,{'$oid': '60202d40b1c4f08d4af2c8e5'},Fruteria Santos,39.940697,-3.991567,'Vegetarian / Vegan Restaurant',"{'type': 'Point', 'coordinates': [-3.991566896...",60.485028
